In [1]:
import numpy as np
import pandas as pd

In [12]:
df = pd.read_csv('F://Projects/player_stats.csv')
df

,Unnamed: 0,match_id,game_id,team,score_team,opponent,score_opp,win_lose,map,map_pick,...,rating,acs,kill,death,assist,kast%,adr,hs%,fk,fd
0,0,53658,119512,ZETA,5,DRX,13,opponent win,Ascent,opponent pick,...,1.19,239,15,11,6,72%,146,16%,1,0
1,1,53658,119512,ZETA,5,DRX,13,opponent win,Ascent,opponent pick,...,0.84,234,15,14,6,61%,145,32%,1,4
2,2,53658,119512,ZETA,5,DRX,13,opponent win,Ascent,opponent pick,...,0.68,173,10,15,2,67%,123,18%,4,2
3,3,53658,119512,ZETA,5,DRX,13,opponent win,Ascent,opponent pick,...,0.63,142,8,15,4,83%,92,29%,3,2
4,4,53658,119512,ZETA,5,DRX,13,opponent win,Ascent,opponent pick,...,0.57,114,7,16,8,61%,63,13%,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6225,6225,61192,137642,EG,13,PRX,10,team win,Lotus,team pick,...,1.43,244,23,13,10,83%,162,46%,0,1
6226,6226,61192,137642,EG,13,PRX,10,team win,Lotus,team pick,...,1.26,338,26,18,5,78%,225,18%,7,3
6227,6227,61192,137642,EG,13,PRX,10,team win,Lotus,team pick,...,1.09,198,16,16,1,65%,131,36%,2,2
6228,6228,61192,137642,EG,13,PRX,10,team win,Lotus,team pick,...,0.86,180,13,17,15,74%,108,29%,2,3


In [19]:
# encoding binary data
df_new = df.drop(columns = ['match_id','Unnamed: 0','game_id'])
df_new = df_new.assign(team_win = df['win_lose']=='team win').drop(columns=['win_lose'])
df_new = df_new.assign(map_pick = df_new['map_pick'] == 'team pick')
df_new

,team,score_team,opponent,score_opp,map,map_pick,player_id,player,agent,rating,acs,kill,death,assist,kast%,adr,hs%,fk,fd,team_win
0,ZETA,5,DRX,13,Ascent,False,999,Laz,sova,1.19,239,15,11,6,72%,146,16%,1,0,False
1,ZETA,5,DRX,13,Ascent,False,1395,TENNN,killjoy,0.84,234,15,14,6,61%,145,32%,1,4,False
2,ZETA,5,DRX,13,Ascent,False,424,Dep,jett,0.68,173,10,15,2,67%,123,18%,4,2,False
3,ZETA,5,DRX,13,Ascent,False,1000,crow,kayo,0.63,142,8,15,4,83%,92,29%,3,2,False
4,ZETA,5,DRX,13,Ascent,False,6668,SugarZ3ro,omen,0.57,114,7,16,8,61%,63,13%,0,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6225,EG,13,PRX,10,Lotus,True,26171,Demon1,astra,1.43,244,23,13,10,83%,162,46%,0,1,True
6226,EG,13,PRX,10,Lotus,True,3993,jawgemo,raze,1.26,338,26,18,5,78%,225,18%,7,3,True
6227,EG,13,PRX,10,Lotus,True,604,Boostio,killjoy,1.09,198,16,16,1,65%,131,36%,2,2,True
6228,EG,13,PRX,10,Lotus,True,11225,Ethan,skye,0.86,180,13,17,15,74%,108,29%,2,3,True


In [24]:
player_dict = {}
for i in df_new['player_id'].unique():
    player_dict[i] = df_new[df_new['player_id'] == i]['player'].iloc[0]
player_dict

{999: 'Laz',
 1395: 'TENNN',
 424: 'Dep',
 1000: 'crow',
 6668: 'SugarZ3ro',
 4462: 'MaKo',
 11600: 'Foxy9',
 485: 'stax',
 772: 'Zest',
 488: 'Rb',
 2489: 'Munchkin',
 31207: 'Carpe',
 474: 'xeta',
 482: 'iNTRO',
 4545: 'Sayaplayer',
 805: 'Bazzi',
 824: 'AYRIN',
 9196: 't3xture',
 6654: 'WRONSKI',
 8497: 'Monyet',
 5129: 'ban',
 7378: 'Jinggg',
 9802: 'Benkai',
 9801: 'f0rsakeN',
 9803: 'd4v41',
 9800: 'mindfreak',
 4559: 'Suggest',
 1002: 'takej',
 1003: 'Reita',
 6700: 'Anthem',
 7718: 'xnfri',
 8504: 'invy',
 2333: 'BORKUM',
 7405: 'Jremy',
 2679: 'DubsteP',
 2334: 'JessieVash',
 13788: 'JitBoyS',
 2452: 'Patiphan',
 18035: 'garnetS',
 6723: 'foxz',
 3977: 'Crws',
 3342: 'Sushiboys',
 4871: 'xffero',
 9735: '2ge',
 7045: 'fl1pzjder',
 7403: 'EJAY',
 7764: 'Lmemore',
 13039: 'Meteor',
 4827: 'GodDead',
 771: 'k1Ng',
 1960: 'eKo',
 983: 'TS',
 4056: 'Sylvan',
 804: 'BuZz',
 17086: 'something',
 9747: 'Emman',
 10584: 'Seoldam',
 3626: 'Lightningfast',
 4095: 'SkRossi',
 24679: 'cgrs

In [28]:
df_new['agent'].unique()

array(['sova', 'killjoy', 'jett', 'kayo', 'omen', 'viper', 'skye',
       'harbor', 'fade', 'astra', 'breach', 'raze', 'phoenix', 'yoru',
       'cypher', 'sage', 'brimstone', 'neon', 'gekko', 'reyna', 'chamber'],
      dtype=object)

In [27]:
pd.get_dummies(df_new, columns=['map'])

,team,score_team,opponent,score_opp,map_pick,player_id,player,agent,rating,acs,...,fd,team_win,map_Ascent,map_Bind,map_Fracture,map_Haven,map_Icebox,map_Lotus,map_Pearl,map_Split
0,ZETA,5,DRX,13,False,999,Laz,sova,1.19,239,...,0,False,True,False,False,False,False,False,False,False
1,ZETA,5,DRX,13,False,1395,TENNN,killjoy,0.84,234,...,4,False,True,False,False,False,False,False,False,False
2,ZETA,5,DRX,13,False,424,Dep,jett,0.68,173,...,2,False,True,False,False,False,False,False,False,False
3,ZETA,5,DRX,13,False,1000,crow,kayo,0.63,142,...,2,False,True,False,False,False,False,False,False,False
4,ZETA,5,DRX,13,False,6668,SugarZ3ro,omen,0.57,114,...,1,False,True,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6225,EG,13,PRX,10,True,26171,Demon1,astra,1.43,244,...,1,True,False,False,False,False,False,True,False,False
6226,EG,13,PRX,10,True,3993,jawgemo,raze,1.26,338,...,3,True,False,False,False,False,False,True,False,False
6227,EG,13,PRX,10,True,604,Boostio,killjoy,1.09,198,...,2,True,False,False,False,False,False,True,False,False
6228,EG,13,PRX,10,True,11225,Ethan,skye,0.86,180,...,3,True,False,False,False,False,False,True,False,False
